In [1]:
import pathlib
import cdmetadl.helpers.general_helpers
import cdmetadl.dataset
import cdmetadl.samplers
import cdmetadl.augmentation
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

%load_ext autoreload
%autoreload 2
%matplotlib inline  

/fastdata/vilab25/cd-metadl-cyLvmR8c-py3.11/lib/python3.11/site-packages/controlnet_aux/mediapipe_face/mediapipe_face_common.py:7: UserWarning: The module 'mediapipe' is not installed. The package will have limited functionality. Please install it using the command: pip install 'mediapipe'
  warnings.warn(
/fastdata/vilab25/cd-metadl-cyLvmR8c-py3.11/lib/python3.11/site-packages/controlnet_aux/segment_anything/modeling/tiny_vit_sam.py:654: UserWarning: Overwriting tiny_vit_5m_224 in registry with controlnet_aux.segment_anything.modeling.tiny_vit_sam.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/fastdata/vilab25/cd-metadl-cyLvmR8c-py3.11/lib/python3.11/site-packages/controlnet_aux/segment_anything/modeling/tiny_vit_sam.py:654: UserWarning: Overwriting tiny_vit_11m_224 in registry with controlnet_aux.segment_anything.modeling.tiny_vit_sam.tiny_vit_11m_224. This is becau

In [12]:
path_to_dataset = "/fastdata/vilab24/meta-album"


for dataset_name in ["ACT_410", "AWA", "BTS", "FNG", "PRT", "BRD", "BCT"]:
    dataset_path = pathlib.Path(path_to_dataset)
    dataset_info_dict = cdmetadl.helpers.general_helpers.check_datasets(dataset_path, [dataset_name])

    dataset = cdmetadl.dataset.ImageDataset(dataset_name, dataset_info_dict[dataset_name])

    n_way_sampler = cdmetadl.samplers.ValueSampler(value=10)
    k_shot_sampler = cdmetadl.samplers.ValueSampler(value=10)

    task = dataset.generate_task(n_way_sampler, k_shot_sampler, query_size=4)

    augmentation_size = {"threshold": 0.75,
                        "scale": 1,
                        "offset": 5,
                        "maximum": 15}

    device = "cuda:3"

    all_images = []

    generative_augmentor = cdmetadl.augmentation.GenerativeAugmentation(augmentation_size,
                                                                        device=device,
                                                                        annotator_type="hed",
                                                                        keep_original_data=False,
                                                                        cache_images=True,
                                                                        guessing_mode=False)

    generative_augmentor.augment(task.support_set, conf_scores=task.number_of_ways*[0.1])

    for i, item in enumerate(generative_augmentor.generated_images):
            fig, axes = plt.subplots(1, 2, figsize=(10, 5))
            
            axes[0].imshow(item['original_image'])
            axes[0].set_title('Original Image')
            axes[0].axis('off')
            
            axes[1].imshow(item['generated_image'])
            axes[1].set_title('Generated Image')
            axes[1].axis('off')

            # Create the directory if it doesn't exist
            directory = '/visinf/home/vilab25/examples_experiments/failure_cases/'
            
            plt.tight_layout()

            # Save the figure to the specified path
            fig.savefig(os.path.join(directory, f'example_{dataset_name}_{i}_{item['classname']}.png'))
            
            plt.close(fig)  # Close the figure to free up memory

    print("Images saved successfully.")


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]